In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.impute import KNNImputer

In [ ]:
vc1 = pd.read_csv('../input/solar-gen/m2015.csv', error_bad_lines=False)
vc2 = pd.read_csv('../input/solar-gen/m2016.csv', error_bad_lines=False)
vc3 = pd.read_csv('../input/solar-gen/m2016.csv', error_bad_lines=False)
#vc3['RefCell1_Wm2_Avg']=vc3['RefCell1_Wm2_Avg']/3
#c1['RefCell1_Wm2_Avg']=vc1['RefCell1_Wm2_Avg']/2


In [ ]:
vtnt=pd.concat([vc1,vc2,vc3],axis=0)

In [ ]:
vtnt

In [ ]:
vtnt.isnull().any()

In [ ]:
vtnt.pop('WindSpeedAve_ms')
vtnt.pop('WindRef_V_Min')

In [ ]:
vtnt

In [ ]:
btnt=vtnt.dropna()

In [ ]:
btnt

In [ ]:
X = btnt.iloc[:, 0:-1]
y = btnt.iloc[:, -1]
X = X.values
y = y.values



In [ ]:
y

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=False)

In [ ]:
y_test

In [ ]:
y_train

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
"""import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
ftf.keras.layers.Dropout(
    rate, noise_shape=None, seed=None, **kwargs
)"""

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test=np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
X_train.shape[1]


In [ ]:
import tensorflow as tf
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding


def make_model(batch_size=None):
    source = Input(shape=(X_train.shape[1],), batch_size=batch_size,
                   dtype=tf.int32, name='Input')
    embedding = Embedding(input_dim=11,
                          output_dim=128, name='Embedding')(source)
    lstm = LSTM(32, name='LSTM')(embedding)
    predicted_var = Dense(1, activation='sigmoid', name='Output')(lstm)
    model = tf.keras.Model(inputs=[source], outputs=[predicted_var])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='binary_crossentropy',
        metrics=['acc'])
    return model


training_model = make_model(batch_size=128)

import os
import tensorflow as tf
import tensorflow as tf
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    def make_model(batch_size=None):
        source = Input(shape=(X_train.shape[1],), batch_size=batch_size,
                       dtype=tf.int32, name='Input')
        embedding = Embedding(input_dim=11,
                              output_dim=128, name='Embedding')(source)
        lstm = LSTM(32, name='LSTM')(embedding)
        predicted_var = Dense(1, activation='sigmoid', name='Output')(lstm)
        model = tf.keras.Model(inputs=[source], outputs=[predicted_var])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
            loss='binary_crossentropy',
            metrics=['acc'])
        return model


training_model = make_model(batch_size=128)

# train model normally
model=make_model(128)
history = model.fit(X_train, y_train,
                        epochs=20,
                        batch_size=128 * 8,
                        validation_split=0.2)
model.save_weights('./tpu_model.h5', overwrite=True)
model.evaluate(x_test, y_test, batch_size=128 * 8)

# This address identifies the TPU we'll use when configuring TensorFlow.



history = model.fit(X_train, y_train,
                        epochs=20,
                        batch_size=128 * 8,
                        validation_split=0.2)
model.save_weights('./tpu_model.h5', overwrite=True)
model.evaluate(x_test, y_test, batch_size=128 * 8)


In [ ]:
import tensorflow as tf
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():
    regressor = Sequential()

    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))

    regressor.add(LSTM(units = 50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'root mean squared error')
    #filepath="weights.best.hdf5"
    filepath="weights-improvement-{epoch:02d}.hdf5"

    #checkpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    #checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True, mode='max')
    #callbacks_list = [checkpoint]

#regressor.fit(X_train, y_train,validation_split=0.1, epochs = 15, batch_size = 32,callbacks=callbacks_list)

In [ ]:

#pred = regressor.predict(X_test)



In [ ]:
from keras.models import load_model
model = load_model('../input/weights-questionable/weights-improvement-new-02.hdf5')
filepath="weights-improvement-newy-{epoch:02d}.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs = 10, batch_size = 32,callbacks=callbacks_list,verbose=1)


In [ ]:
loadedmodel = load_model('./weights-improvement-newy-06.hdf5')

In [ ]:
y_pred=loadedmodel.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False)

In [ ]:
y_test.shape

In [ ]:
a=np.sum(y_test, dtype = np.float32)

In [ ]:
y_pred.shape

In [ ]:
np.sum(y_pred, dtype = np.float32)

In [ ]:
X_test[:,-2]

In [ ]:
for i in range(5,40):
    X_test[:,-2]=i
    #X_test=np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    y_pred = loadedmodel.predict(X_test)
    sum=np.sum(y_pred, dtype = np.float32)
    
    if sum>a:
        a=sum
        angle=i
        print(a)
        print(i)